<a href="https://colab.research.google.com/github/tunjis/Moving-Average-Crossover-Signals_Python/blob/main/Moving_Average_Crossover_Signals_Interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import plotly.graph_objects as go
import sys

# List of stocks to check (MAG7 in this case)
stocks_to_check = ['NQ=F', 'GOOGL', 'AMZN', 'AAPL', 'META', 'MSFT', 'NVDA', 'TSLA', 'PLTR']

# Calculate dynamic date range (past 1 year up to last trading day)
start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
end_date = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d') # Use T+1 for end date

# --- Function Definitions ---

def get_stock_data(stock_symbol):
    """Downloads and prepares historical stock data from Yahoo Finance."""
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date, progress=False)
    if not stock_data.empty:
        stock_data.index = pd.to_datetime(stock_data.index)
        # Simplify columns if yfinance returns MultiIndex for a single ticker
        if isinstance(stock_data.columns, pd.MultiIndex):
            try:
                # Attempt to drop the ticker level from columns
                stock_data.columns = stock_data.columns.droplevel(1)
            except Exception as e:
                print(f"     -> Warning: Could not simplify MultiIndex columns for {stock_symbol}: {e}")
    return stock_data

def moving_average_strategy(stock_data):
    """Calculates MAs and generates buy/sell signals."""
    required_data_points_ma50 = 50
    required_data_points_ma200 = 200

    # Initialize columns
    stock_data['MA_50'] = pd.NA
    stock_data['MA_200'] = pd.NA
    stock_data['Buy_Signal'] = False
    stock_data['Sell_Signal'] = False

    # Basic data validation
    if 'Close' not in stock_data.columns or stock_data.empty:
        return stock_data # Return early if essential data is missing

    close_count = int(stock_data['Close'].count()) # Ensure count is integer

    # Calculate MAs if enough data exists
    if close_count >= required_data_points_ma50:
        stock_data['MA_50'] = stock_data['Close'].rolling(window=required_data_points_ma50).mean()
        if close_count >= required_data_points_ma200:
            stock_data['MA_200'] = stock_data['Close'].rolling(window=required_data_points_ma200).mean()

    # Generate signals robustly if MA50 was calculated
    if stock_data['MA_50'].notna().any():
        # Conditions check for valid MA values before comparison
        buy_condition = (stock_data['MA_50'] > stock_data['MA_200']) & stock_data['MA_50'].notna() & stock_data['MA_200'].notna()
        sell_condition = (stock_data['MA_50'] < stock_data['MA_200']) & stock_data['MA_50'].notna() & stock_data['MA_200'].notna()
        stock_data.loc[buy_condition, 'Buy_Signal'] = True
        stock_data.loc[sell_condition, 'Sell_Signal'] = True

    return stock_data

def plot_stock_chart_plotly(df, stock_symbol):
    """Generates an interactive stock chart using Plotly and displays it inline."""

    # --- Data Validation ---
    required_cols = ['Close', 'MA_50', 'MA_200', 'Buy_Signal', 'Sell_Signal']
    if not all(col in df.columns for col in required_cols) or df['Close'].isnull().all():
        print(f"  -> Plotting skipped for {stock_symbol}: Missing data or required columns.")
        return

    fig = go.Figure()

    # --- Add Price and MA Lines ---
    # Add Close Price trace with its hover template
    fig.add_trace(go.Scatter(
        x=df.index, y=df['Close'], mode='lines', name='Close Price',
        line=dict(color='black', width=1.5),
        hovertemplate='Closing Price: %{y:.2f}<br>Date: %{x|%d %b \'%y}<extra></extra>' # Updated hover for Close Price
    ))
    # Plot MAs only if they contain valid numbers, add specific hover info
    if df['MA_50'].notna().any():
        fig.add_trace(go.Scatter(
            x=df.index, y=df['MA_50'], mode='lines', name='50-day MA',
            line=dict(color='orange', width=1, dash='dash'),
            # hoverinfo='skip' # Removed hoverinfo='skip'
            hovertemplate='MA50: %{y:.2f}<br>Date: %{x|%d %b \'%y}<extra></extra>' # Added hovertemplate
        ))
    if df['MA_200'].notna().any():
        fig.add_trace(go.Scatter(
            x=df.index, y=df['MA_200'], mode='lines', name='200-day MA',
            line=dict(color='blue', width=1, dash='dash'),
            # hoverinfo='skip' # Removed hoverinfo='skip'
            hovertemplate='MA200: %{y:.2f}<br>Date: %{x|%d %b \'%y}<extra></extra>' # Added hovertemplate
        ))

    # --- Add Signal Markers ---
    buy_signals = df[df['Buy_Signal']]
    sell_signals = df[df['Sell_Signal']]

    # Add markers with specific hovertemplate
    if not buy_signals.empty:
        fig.add_trace(go.Scatter(
            x=buy_signals.index, y=buy_signals['Close'], mode='markers', name='Buy Signal',
            marker=dict(symbol='triangle-up', color='green', size=8),
            hovertemplate='Closing Price: %{y:.2f}<br>Date: %{x|%d %b \'%y}<extra></extra>'
        ))
    if not sell_signals.empty:
        fig.add_trace(go.Scatter(
            x=sell_signals.index, y=sell_signals['Close'], mode='markers', name='Sell Signal',
            marker=dict(symbol='triangle-down', color='red', size=8),
            hovertemplate='Closing Price: %{y:.2f}<br>Date: %{x|%d %b \'%y}<extra></extra>'
        ))

    # --- Customize Layout ---
    fig.update_layout(
        title=f'{stock_symbol} - Moving Average Signals (Interactive)',
        xaxis_title='Date', yaxis_title='Stock Price', legend_title='Legend',
        hovermode='closest', # Keep hovermode='closest'
        xaxis_rangeslider_visible=False
    )

    # --- Display Plot Inline ---
    try:
        fig.show()
    except Exception as show_e:
        print(f"  -> ERROR: Failed to display chart for {stock_symbol}: {show_e}")


# --- Main Execution Loop ---
print("Starting stock analysis...")
for stock_symbol in stocks_to_check:
    print(f"\n===== Processing {stock_symbol} =====")
    try:
        stock_data = get_stock_data(stock_symbol)
        if stock_data.empty:
            continue

        strategy_results = moving_average_strategy(stock_data.copy())
        plot_stock_chart_plotly(strategy_results, stock_symbol)

    except Exception as e:
        print(f"\n*** An error occurred processing {stock_symbol}: {e} ***")
        # import traceback # Uncomment for detailed debugging
        # traceback.print_exc()

print("\n===== Processing complete. =====")


Starting stock analysis...

===== Processing NQ=F =====


/tmp/ipython-input-1639623194.py:18: FutureWarning:

YF.download() has changed argument auto_adjust default to True




===== Processing GOOGL =====


/tmp/ipython-input-1639623194.py:18: FutureWarning:

YF.download() has changed argument auto_adjust default to True




===== Processing AMZN =====


...


===== Processing NVDA =====


/tmp/ipython-input-1639623194.py:18: FutureWarning:

YF.download() has changed argument auto_adjust default to True




===== Processing TSLA =====


/tmp/ipython-input-1639623194.py:18: FutureWarning:

YF.download() has changed argument auto_adjust default to True




===== Processing complete. =====
